In [96]:
import numpy as np
import pandas as pd
import setuptools.dist
import plotly.graph_objects as go
import plotly.express as px
from keras import layers as L
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import regularizers

In [97]:
df = pd.read_csv("../data/people_data.csv")
print(df.head())
df = df.drop("img_name",axis=1)

   age  ethnicity  gender                        img_name  \
0    1          2       0  20161219203650636.jpg.chip.jpg   
1    1          2       0  20161219222752047.jpg.chip.jpg   
2    1          2       0  20161219222832191.jpg.chip.jpg   
3    1          2       0  20161220144911423.jpg.chip.jpg   
4    1          2       0  20161220144914327.jpg.chip.jpg   

                                              pixels  
0  129 128 128 126 127 130 133 135 139 142 145 14...  
1  164 74 111 168 169 171 175 182 184 188 193 199...  
2  67 70 71 70 69 67 70 79 90 103 116 132 145 155...  
3  193 197 198 200 199 200 202 203 204 205 208 21...  
4  202 205 209 210 209 209 210 211 212 214 218 21...  


In [98]:
df["pixels"] = df["pixels"].apply(lambda x: np.array(x.split(),dtype="float32"))
df["pixels"] = df["pixels"] / 255

In [99]:
x_train = np.zeros(shape=(23705,2304))

for indexing in range(len(df["pixels"])):
    x_train[indexing] = df["pixels"][indexing]

In [100]:
x_train = x_train.reshape(-1,48,48,1)

In [101]:
x = x_train
y = df.iloc[:, :-1].values

In [102]:
x_train,x_test,y_train,y_test = train_test_split(x, y, train_size=0.8, random_state=35, shuffle=True)

In [103]:
early_stopper = tf.keras.callbacks.EarlyStopping(
    monitor="loss",
    patience=5,
    mode="min"
    )

# checkpoint_model = tf.keras.callbacks.ModelCheckpoint(
#     monitor="val_accuracy",
#     save_best_only=True,
#     save_weights_only=False,
#     filepath="./.keras"
#     )

In [104]:
COMPILE_OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.0001)
COMPILE_LOSS = "msle"
COMPILE_METRICS = ["accuracy"]
INPUT_SHAPE = (x_train.shape[1],x_train.shape[2],x_train.shape[3])
OUTPUT_CLASS = 3

In [105]:
model = tf.keras.Sequential()

model.add(L.Input(shape=INPUT_SHAPE))
model.add(L.Conv2D(32, (3, 3), activation="relu"))
model.add(L.BatchNormalization())
model.add(L.MaxPooling2D((2, 2)))

model.add(L.Conv2D(64,(3,3),activation="relu"))
model.add(L.Dropout(0.3))
model.add(L.MaxPooling2D((2,2)))
model.add(L.BatchNormalization())

model.add(L.Conv2D(128,(3,3),activation="relu"))
model.add(L.Dropout(0.3))
model.add(L.MaxPooling2D((2,2)))
model.add(L.BatchNormalization())


model.add(L.Flatten())
model.add(L.Dense(128,activation="relu"))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.5))
model.add(L.Dense(64,activation="relu"))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.5))
model.add(L.Dense(OUTPUT_CLASS,activation="softmax"))

In [106]:
model.compile(optimizer=COMPILE_OPTIMIZER,loss=COMPILE_LOSS,metrics=COMPILE_METRICS)

In [107]:
path = '../models/combined_model.h5'

cnn_model = model.fit(
    x_train, y_train,
    validation_data=(
        x_test, 
        y_test),
    callbacks=[early_stopper], # checkpoint_model
    batch_size=12,
    epochs=30
    )

Epoch 1/30
1581/1581 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.4946 - loss: 3.3666 - val_accuracy: 0.9515 - val_loss: 2.8298
Epoch 2/30
1581/1581 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.7972 - loss: 3.0228 - val_accuracy: 0.9580 - val_loss: 2.8122
Epoch 3/30
1581/1581 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.8560 - loss: 2.9241 - val_accuracy: 0.9547 - val_loss: 2.8088
Epoch 4/30
1581/1581 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.8994 - loss: 2.8649 - val_accuracy: 0.9584 - val_loss: 2.8067
Epoch 5/30
1581/1581 ━━━━━━━━━━━━━━━━━━━━ 29s 18ms/step - accuracy: 0.9453 - loss: 2.8368 - val_accuracy: 0.9612 - val_loss: 2.8059
Epoch 6/30
1581/1581 ━━━━━━━━━━━━━━━━━━━━ 43s 27ms/step - accuracy: 0.9525 - loss: 2.8261 - val_accuracy: 0.9608 - val_loss: 2.8060
Epoch 7/30
1581/1581 ━━━━━━━━━━━━━━━━━━━━ 48s 31ms/step - accuracy: 0.9564 - loss: 2.8113 - val_accuracy: 0.9610 - val_loss: 2.8059
Epoch 8/30
1581/1581 ━━━━━━━━━━━━━━━━━━━━ 45s 28ms/step - accuracy: 0.9546 -